In [1]:
#Importing numpy, pandas and matplotlip
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
df=pd.read_csv('cleaned_data2.csv')
df

,target,text,user_mentions
0,0,aww bummer shoulda get david carr third day,0.000000
1,0,upset can update facebook texting might cry r...,0.000515
2,0,dive many time ball manage save rest go bound,0.009004
3,0,whole body feel itchy like fire,0.000257
4,0,behave mad can see,0.001801
...,...,...,...
1592814,1,wake school well feel ever,0.000000
1592815,1,thewdb com cool hear old walt interviews bmta,0.000000
1592816,1,ready mojo makeover ask detail,0.000000
1592817,1,happy th birthday boo time tupac amaru shakur,0.000515


In [3]:
from scipy.sparse import csr_matrix
from scipy.sparse import hstack

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# Create an instance of CountVectorizer
vectorizer = CountVectorizer()
vectorizer2 = TfidfVectorizer()
count_vect_matrix=vectorizer.fit_transform(df['text'])
tfidf_vect_matrix=vectorizer2.fit_transform(df['text'])



In [13]:
count_vect_matrix

<1592819x242934 sparse matrix of type '<class 'numpy.int64'>'
	with 10664380 stored elements in Compressed Sparse Row format>

In [5]:
labels=df['target']
labels

0          0
1          0
2          0
3          0
4          0
          ..
1592814    1
1592815    1
1592816    1
1592817    1
1592818    1
Name: target, Length: 1592819, dtype: int64

In [6]:
user_mentions = pd.Series(df['user_mentions']).values.reshape(-1, 1)
user_mentions_sparse = csr_matrix(user_mentions)
count_vect_hstack = hstack([count_vect_matrix, user_mentions_sparse]) 
tfidf_vect_hstack = hstack([tfidf_vect_matrix, user_mentions_sparse]) 

In [23]:
count_vect_hstack

<1592819x242935 sparse matrix of type '<class 'numpy.float64'>'
	with 11344231 stored elements in Compressed Sparse Row format>

In [24]:
tfidf_vect_hstack

<1592819x242935 sparse matrix of type '<class 'numpy.float64'>'
	with 11344231 stored elements in Compressed Sparse Row format>

In [7]:
from sklearn.model_selection import train_test_split


In [8]:
from sklearn.linear_model import LogisticRegression

In [9]:
from sklearn.metrics import root_mean_squared_error
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score
def evaluate(y_test,predicitions):
    mse = root_mean_squared_error(y_test, predicitions)
    print("Mean Squared Error: ",mse)
    pre = precision_score(y_test, predicitions)
    print("Precision: ",pre)
    f1 = f1_score(y_test, predicitions)
    print("F1 Score: ",f1)
    acc = accuracy_score(y_test, predicitions)
    print("Accuracy: ",acc)
    r2 = r2_score(y_test, predicitions)
    print("R2 Score: ",r2)


In [10]:
X_train_cv, X_test_cv, y_train_cv, y_test_cv = train_test_split(count_vect_hstack, labels, test_size=0.2, random_state=42)
X_train_tfidf, X_test_tfidf, y_train_tfidf, y_test_tfidf = train_test_split(tfidf_vect_hstack, labels, test_size=0.2, random_state=42)

In [13]:
import warnings

# Disable all warnings
warnings.filterwarnings("ignore")

In [11]:
from sklearn.model_selection import GridSearchCV


In [77]:
model = LogisticRegression()
#max_iter=300, penalty='l2', C=1, solver='saga'

param_grid = {
    'penalty': ['l2'],           # Regularization technique
    'C': [0.01, 0.1, 1, 10],      # Inverse of regularization strength
    'max_iter': [300],         # Maximum number of iterations
    #'multi_class': ['ovr','multinomial']
    'solver': ['lbfgs','liblinear','newton-cg','sag','saga']
}
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, verbose=10,n_jobs=5)


In [78]:
grid_search.fit(X_train_cv, y_train_cv)


Fitting 3 folds for each of 20 candidates, totalling 60 fits


c:\Python312\Lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


GridSearchCV(cv=3, estimator=LogisticRegression(), n_jobs=5,
             param_grid={'C': [0.01, 0.1, 1, 10], 'max_iter': [300],
                         'penalty': ['l2'],
                         'solver': ['lbfgs', 'liblinear', 'newton-cg', 'sag',
                                    'saga']},
             verbose=10)

In [79]:
best_model = grid_search.best_estimator_
predictions = best_model.predict(X_test_cv)
best_model.get_params()

In [80]:
print('CountVectorizor: ')
evaluate(y_test_cv,predictions)

CountVectorizor: 
Mean Squared Error:  0.4754755731160934
Precision:  0.7633689279105005
F1 Score:  0.7797176274836057
Accuracy:  0.7739229793699225
R2 Score:  0.09567499634111154


In [81]:
best_model.get_params()

{'C': 1,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 300,
 'multi_class': 'deprecated',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'saga',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [14]:
model = LogisticRegression(max_iter=300, penalty='l2', C=1, solver='saga',n_jobs=5,verbose=10)
model.fit(X_train_cv, y_train_cv)
predictions = model.predict(X_test_cv)
print('CountVectorizor: ')
evaluate(y_test_cv,predictions)

[Parallel(n_jobs=5)]: Using backend ThreadingBackend with 5 concurrent workers.


max_iter reached after 230 seconds
CountVectorizor: 
Mean Squared Error:  0.475485476003244
Precision:  0.7633678309616214
F1 Score:  0.7797050825995063


[Parallel(n_jobs=5)]: Done   1 tasks      | elapsed:  3.8min


Accuracy:  0.7739135621099685
R2 Score:  0.09563732659644375


In [15]:
model.fit(X_train_tfidf, y_train_tfidf)
predictions = model.predict(X_test_tfidf)
print('TFIDFVectorizor: ')
evaluate(y_test_cv,predictions)

[Parallel(n_jobs=5)]: Using backend ThreadingBackend with 5 concurrent workers.


convergence after 22 epochs took 16 seconds
TFIDFVectorizor: 
Mean Squared Error:  0.47385532409261477
Precision:  0.7664383107688785
F1 Score:  0.7805424311222925


[Parallel(n_jobs=5)]: Done   1 tasks      | elapsed:   16.0s


Accuracy:  0.775461131829083
R2 Score:  0.1018277213035228


In [16]:
from sklearn.linear_model import LinearRegression

In [18]:
model = LinearRegression(copy_X=True, fit_intercept=True, n_jobs=5, positive=False)
#{'copy_X': True, 'fit_intercept': True, 'n_jobs': None, 'positive': False}


In [ ]:
model = LinearRegression()
param_grid = {
    'fit_intercept': [True, False],
    'copy_X': [True, False],
}

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, verbose=10)
grid_search.fit(X_train_cv, y_train_cv)
best_model = grid_search.best_estimator_
predictions = best_model.predict(X_test_cv)
predictions2 = (predictions > 0.5).astype(int)
print('CountVectorizer: ')
evaluate(y_test_cv, predictions2)
print("Best parameters found: ", best_model.get_params())

CountVectorizer: 
Mean Squared Error:  0.4304197655667803
R2 Score:  0.2589414354341024
Best parameters found:  {'copy_X': True, 'fit_intercept': True, 'n_jobs': None, 'positive': False}

In [19]:
model.fit(X_train_cv, y_train_cv)
predictions = model.predict(X_test_cv)
print('CountVectorizor: ')
mse = root_mean_squared_error(y_test_cv, predictions)
print("Mean Squared Error: ",mse)
r2 = r2_score(y_test_cv, predictions)
print("R2 Score: ",r2)

CountVectorizor: 
Mean Squared Error:  0.43042207679729727
R2 Score:  0.25893347686944146


In [20]:
model.fit(X_train_tfidf, y_train_tfidf)
predictions = model.predict(X_test_tfidf)
print('TFIDFVectorizor: ')
predictions2 = (predictions > 0.5).astype(int)
evaluate(y_test_cv, predictions2)

TFIDFVectorizor: 
Mean Squared Error:  0.42212927148638
R2 Score:  0.28721417543490035


In [22]:
from sklearn.neighbors import KNeighborsClassifier
param_grid = {
    'n_neighbors': [1,3,5,7,9,11,13],           # Regularization technique
    'weights': ['uniform','distance'],      # Inverse of regularization strength
    'algorithm': ['brute'],         # Maximum number of iterations
    #'multi_class': ['ovr','multinomial']
    'metric': ['euclidean','minkowski','manhattan']
}
knn = KNeighborsClassifier()
grid_search = GridSearchCV(estimator=knn, param_grid=param_grid, cv=3, verbose=10)


# for _ in tqdm(range(1), desc="Training Progress"):
#     knn.fit(X_train_cv, y_train_cv)


In [ ]:
grid_search.fit(X_train_cv, y_train_cv)
best_model = grid_search.best_estimator_
predictions = best_model.predict(X_test_cv)
best_model.get_params()
print('CountVectorizor: ')
evaluate(y_test_cv,predictions)

Fitting 3 folds for each of 42 candidates, totalling 126 fits
[CV 1/3; 1/42] START algorithm=brute, metric=euclidean, n_neighbors=1, weights=uniform


In [29]:
knn = KNeighborsClassifier()
param_grid = {
    'n_neighbors': [5],           # Regularization technique
}
grid_search = GridSearchCV(estimator=knn, param_grid=param_grid, cv=2, verbose=10)


In [30]:
grid_search.fit(X_train_cv, y_train_cv)
best_model = grid_search.best_estimator_
predictions = best_model.predict(X_test_cv)


Fitting 2 folds for each of 1 candidates, totalling 2 fits
[CV 1/2; 1/1] START n_neighbors=5...............................................
[CV 1/2; 1/1] END ...............n_neighbors=5;, score=0.695 total time=167.8min
[CV 2/2; 1/1] START n_neighbors=5...............................................
[CV 2/2; 1/1] END ...............n_neighbors=5;, score=0.695 total time=164.4min
CountVectorizor: 
Mean Squared Error:  0.5452721625656952
R2 Score:  -0.18930917865253094


In [34]:
print('CountVectorizor: ')
evaluate(y_test_cv,predictions)

Mean Squared Error:  0.5452721625656952
Precision:  0.6911741441034985
F1 Score:  0.7135373820469393
Accuracy:  0.7026782687309301
R2 Score:  -0.18930917865253094


In [ ]:
grid_search.fit(X_train_tfidf, y_train_tfidf)
best_model = grid_search.best_estimator_
predictions = best_model.predict(X_test_tfidf)
print('CountVectorizor: ')
evaluate(y_test_cv,predictions)

In [12]:
from sklearn.naive_bayes import BernoulliNB, MultinomialNB

In [78]:
model = BernoulliNB()
param_grid = {
    'alpha': [0.1, 0.5, 1.0, 2.0, 5.0, 10.0],  # Additive (Laplace/Lidstone) smoothing parameter
    'fit_prior': [True, False],                # Whether to learn class prior probabilities
    'binarize': [0.0, 0.1, 0.5, 1.0]          # Threshold for binarizing the feature
}

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, verbose=10, n_jobs=5)
grid_search.fit(X_train_cv, y_train_cv)
best_model = grid_search.best_estimator_
predictions = best_model.predict(X_test_cv)
print('CountVectorizer: ')
evaluate(y_test_cv, predictions)
print("Best parameters found: ", best_model.get_params())

CountVectorizor: 
Mean Squared Error:  0.48401125782717513
Precision:  0.7676069587467232
F1 Score:  0.7663645613211238
Accuracy:  0.7657331022965558
R2 Score:  0.06291487506166094


Mean Squared Error:  0.48327133780820986
Precision:  0.7684700626231442
F1 Score:  0.7670141574574822
Accuracy:  0.7664488140530631
R2 Score:  0.06577777565641552
Best parameters found:  {'alpha': 5.0, 'binarize': 0.0, 'class_prior': None, 'fit_prior': False, 'force_alpha': True}

In [ ]:
model = MultinomialNB()

param_grid = {
    'alpha': [0.1, 0.5, 1.0, 2.0, 5.0, 10.0],
    'fit_prior': [True, False],
    'force_alpha': [True, False]
}

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, verbose=3, n_jobs=5)
grid_search.fit(X_train_cv, y_train_cv)
best_model = grid_search.best_estimator_
predictions = best_model.predict(X_test_cv)
print('CountVectorizer: ')
evaluate(y_test_cv, predictions)
print("Best parameters found: ", best_model.get_params())

Mean Squared Error:  0.48530016916407587
Precision:  0.7768348324859862
F1 Score:  0.7605961881483514
Accuracy:  0.7644837458093193
R2 Score:  0.0579173556023963
Best parameters found:  {'alpha': 5.0, 'class_prior': None, 'fit_prior': False, 'force_alpha': True}

In [15]:
from sklearn.tree import DecisionTreeClassifier

In [16]:
model = DecisionTreeClassifier(max_depth=10)
param_grid = {
    'criterion': ['gini', 'entropy','log_loss'],
    'splitter': ['best', 'random'],
    'max_features': ['sqrt','log2',None]

}

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, verbose=10)
grid_search.fit(X_train_cv, y_train_cv)
best_model = grid_search.best_estimator_
predictions = best_model.predict(X_test_cv)
print('CountVectorizer: ')
evaluate2(y_test_cv, predictions)
print("Best parameters found: ", best_model.get_params())

Training Progress: 100%|██████████| 1/1 [06:13<00:00, 373.68s/it]


CountVectorizor: 
Mean Squared Error:  0.6310279556486665
Precision:  0.5633206641022308
F1 Score:  0.6990464938065989
Accuracy:  0.6018037191898645
R2 Score:  -0.5928149269527028


F1 Score:  0.6990270531882335
Accuracy:  0.601772328323351
R2 Score:  -0.5929404927682622
Best parameters found:  {'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': 10, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'monotonic_cst': None, 'random_state': None, 'splitter': 'best'}


In [17]:
#Trr=DecisionTreeClassifier(max_depth=10, random_state=42)
#Best parameters found:  {'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': 10, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'monotonic_cst': None, 'random_state': None, 'splitter': 'best'}

# for _ in tqdm(range(1), desc="Training Progress"):
#     Trr.fit(X_train_tfidf, y_train_tfidf)
# predictions = Trr.predict(X_test_tfidf)
# print('TFIDFVectorizor: ')
# evaluate(y_test_tfidf,predictions)

Training Progress: 100%|██████████| 1/1 [06:32<00:00, 392.26s/it]


TFIDFVectorizor: 
Mean Squared Error:  0.6312617166875892
Precision:  0.5631593038821955
F1 Score:  0.6987691643273591
Accuracy:  0.6015086450446379
R2 Score:  -0.5939952456189612


In [19]:
from sklearn.ensemble import RandomForestClassifier

In [20]:
model = RandomForestClassifier(max_depth=10)
param_grid = {
    'criterion': ['gini', 'entropy','log_loss'],
    'max_features': ['sqrt',1000],
    'n_estimators':[300]

}

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, verbose=10)
grid_search.fit(X_train_cv, y_train_cv)
best_model = grid_search.best_estimator_
predictions = best_model.predict(X_test_cv)
print('CountVectorizer: ')
evaluate2(y_test_cv, predictions)
print("Best parameters found: ", best_model.get_params())

Training Progress: 100%|██████████| 1/1 [09:01<00:00, 541.33s/it]


CountVectorizor: 
Mean Squared Error:  0.6312617166875892
Precision:  0.5631593038821955
F1 Score:  0.6987691643273591
Accuracy:  0.6015086450446379
R2 Score:  -0.5939952456189612


Mean Squared Error:  0.5115564081209615
Precision:  0.7393638295213099
F1 Score:  0.7394770477919691
Accuracy:  0.7383100413103804
R2 Score:  -0.046779421411041744
Best parameters found:  {'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'entropy', 'max_depth': 10, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'monotonic_cst': None, 'n_estimators': 300, 'n_jobs': None, 'oob_score': False, 'random_state': None, 'verbose': 0, 'warm_start': False}

In [83]:
from sklearn.svm import LinearSVC

In [84]:
# LSVC=LinearSVC()
# #{'C': 0.01, 'class_weight': None, 'dual': 'auto', 'fit_intercept': True, 'intercept_scaling': 1, 'loss': 'squared_hinge', 'max_iter': 1000, 'multi_class': 'ovr', 'penalty': 'l2', 'random_state': None, 'tol': 0.0001, 'verbose': 0}
# LSVC.fit(X_train_cv, y_train_cv)
# predictions = LSVC.predict(X_test_cv)
# print('CountVectorizor: ')
# evaluate(y_test_cv,predictions)

CountVectorizor: 
Mean Squared Error:  0.48311217196529094
Precision:  0.7576262720483363
F1 Score:  0.7720242840497946
Accuracy:  0.7666026292989792
R2 Score:  0.06639304815265656


In [ ]:
model = LinearSVC()

param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],         # Regularization parameter
    'loss': ['hinge', 'squared_hinge'],   # Loss functions for the SVM
}

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, verbose=10)
grid_search.fit(X_train_cv, y_train_cv)
best_model = grid_search.best_estimator_
predictions = best_model.predict(X_test_cv)
print('CountVectorizer: ')
evaluate(y_test_cv, predictions)
print("Best parameters found: ", best_model.get_params())

Mean Squared Error:  0.4768369358775431
Precision:  0.7577922536870376
F1 Score:  0.7803429819108125
Accuracy:  0.7726265365829158
R2 Score:  0.09048912815850785
Best parameters found:  {'C': 0.01, 'class_weight': None, 'dual': 'auto', 'fit_intercept': True, 'intercept_scaling': 1, 'loss': 'squared_hinge', 'max_iter': 1000, 'multi_class': 'ovr', 'penalty': 'l2', 'random_state': None, 'tol': 0.0001, 'verbose': 0}

In [ ]:
from sklearn.linear_model import Perceptron
print("\nPerceptron:")
param_grid = {'max_iter': [500, 1000], 'eta0': [0.01, 0.1, 1.0]}
model=Perceptron()
grid = GridSearchCV(model, param_grid, cv=3)
grid.fit(X_train_cv, y_train_cv)
best_model = grid.best_estimator_
print(f"Best parameters: ",best_model.get_params())
predictions = best_model.predict(X_test_cv)
evaluate(y_test_cv, predictions)
grid.fit(X_train_tfidf, y_train_tfidf)
best_model = grid.best_estimator_
print(f"Best parameters: ",best_model.get_params())
predictions = best_model.predict(X_test_tfidf)
evaluate(y_test_tfidf, predictions)

CountVectorizor:
Mean Squared Error:  0.31076957848344444
Precision:  0.7035229556242447
F1 Score:  0.6803977272727273
Accuracy:  0.6892304215165556
R2 Score:  -0.24310157403818322
TFIDFVectorizor:
Mean Squared Error:  0.30289361007521254
Precision:  0.6978778319472326
F1 Score:  0.698816691793628
Accuracy:  0.6971063899247875
R2 Score:  -0.21159711091432665


In [35]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
model = Sequential()
model.add(Dense(1, input_dim=X_train_cv.shape[1], activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train_cv, y_train_cv, epochs=10, batch_size=32)
predictions = model.predict(X_test_cv)


Epoch 1/10
39821/39821 ━━━━━━━━━━━━━━━━━━━━ 283s 7ms/step - accuracy: 0.7532 - loss: 0.5362
Epoch 2/10
39821/39821 ━━━━━━━━━━━━━━━━━━━━ 288s 7ms/step - accuracy: 0.7780 - loss: 0.4813
Epoch 3/10
39821/39821 ━━━━━━━━━━━━━━━━━━━━ 285s 7ms/step - accuracy: 0.7837 - loss: 0.4738
Epoch 4/10
39821/39821 ━━━━━━━━━━━━━━━━━━━━ 275s 7ms/step - accuracy: 0.7860 - loss: 0.4702
Epoch 5/10
39821/39821 ━━━━━━━━━━━━━━━━━━━━ 282s 7ms/step - accuracy: 0.7887 - loss: 0.4663
Epoch 6/10
39821/39821 ━━━━━━━━━━━━━━━━━━━━ 291s 7ms/step - accuracy: 0.7900 - loss: 0.4641
Epoch 7/10
39821/39821 ━━━━━━━━━━━━━━━━━━━━ 295s 7ms/step - accuracy: 0.7917 - loss: 0.4622
Epoch 8/10
39821/39821 ━━━━━━━━━━━━━━━━━━━━ 272s 7ms/step - accuracy: 0.7915 - loss: 0.4627
Epoch 9/10
39821/39821 ━━━━━━━━━━━━━━━━━━━━ 265s 7ms/step - accuracy: 0.7933 - loss: 0.4598
Epoch 10/10
39821/39821 ━━━━━━━━━━━━━━━━━━━━ 264s 7ms/step - accuracy: 0.7938 - loss: 0.4589
9956/9956 ━━━━━━━━━━━━━━━━━━━━ 50s 5ms/step


In [37]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print('CountVectorizor: ')
loss, accuracy = model.evaluate(X_test_cv, y_test_cv)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")
predictions2 = (predictions > 0.5).astype(int)
precision = precision_score(y_test_cv, predictions2)
recall = recall_score(y_test_cv, predictions2)
f1 = f1_score(y_test_cv, predictions2)
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

CountVectorizor: 
9956/9956 ━━━━━━━━━━━━━━━━━━━━ 48s 5ms/step - accuracy: 0.7703 - loss: 0.5051
Test Loss: 0.5036
Test Accuracy: 0.7706
Precision: 0.7650
Recall: 0.7839
F1 Score: 0.7744


In [40]:
print('CountVectorizor: ')
mse = root_mean_squared_error(y_test_cv, predictions)
print("Mean Squared Error: ",mse)
r2 = r2_score(y_test_cv, predictions)
print("R2 Score: ",r2)

CountVectorizor: 
Mean Squared Error:  0.40182510179888625
R2 Score:  0.35413432121276855


In [43]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(10,), activation='logistic', max_iter=200, random_state=42, 
verbose=True,   # This will now output updates more frequently
early_stopping=True,   # Stops training if validation score stops improving
n_iter_no_change=10    # Number of iterations with no improvement to wait before stopping)
)
mlp.fit(X_train_cv, y_train_cv)
predictions = mlp.predict(X_test_cv)



Iteration 1, loss = 0.51388196
Validation score: 0.773704
Iteration 2, loss = 0.47759346
Validation score: 0.776553
Iteration 3, loss = 0.46618413
Validation score: 0.776953
Iteration 4, loss = 0.45788871
Validation score: 0.778672
Iteration 5, loss = 0.45202855
Validation score: 0.779786
Iteration 6, loss = 0.44751256
Validation score: 0.779786
Iteration 7, loss = 0.44385805
Validation score: 0.780579
Iteration 8, loss = 0.44058735
Validation score: 0.781371
Iteration 9, loss = 0.43752680
Validation score: 0.781457
Iteration 10, loss = 0.43463641
Validation score: 0.781481
Iteration 11, loss = 0.43186900
Validation score: 0.781104
Iteration 12, loss = 0.42917552
Validation score: 0.781104
Iteration 13, loss = 0.42672200
Validation score: 0.781293
Iteration 14, loss = 0.42440809
Validation score: 0.781073
Iteration 15, loss = 0.42210583
Validation score: 0.780602
Iteration 16, loss = 0.41999505
Validation score: 0.781811
Iteration 17, loss = 0.41780823
Validation score: 0.780359
Iterat

In [44]:
print('CountVectorizor: ')
evaluate(y_test_cv,predictions)

CountVectorizor: 
Mean Squared Error:  0.4693089342908099
Precision:  0.7802737605562595
F1 Score:  0.7808635034854958
Accuracy:  0.7797491241948242
R2 Score:  0.11898001170893857


In [ ]:
mlp = MLPClassifier(hidden_layer_sizes=(10,), activation='logistic', max_iter=200, random_state=42, 
verbose=True,   # This will now output updates more frequently
early_stopping=True,   # Stops training if validation score stops improving
n_iter_no_change=10    # Number of iterations with no improvement to wait before stopping)
)
mlp.fit(X_train_tfidf, y_train_tfidf)
predictions = mlp.predict(X_test_tfidf)
print('CountVectorizor: ')
evaluate(y_test_tfidf,predictions)

In [28]:
import xgboost as xgb
model = xgb.XGBClassifier(objective='binary:logistic', eval_metric='logloss', use_label_encoder=False)
model.fit(X_train_cv, y_train_cv)
y_pred = model.predict(X_test_cv)


In [29]:
print('CountVectorizor: ')
evaluate(y_test_cv,predictions)

CountVectorizor: 
Mean Squared Error:  0.46959646214296763
Precision:  0.7847146855543496
F1 Score:  0.7787638489106679
Accuracy:  0.7794791627428084
R2 Score:  0.11790014569512763


In [ ]:
model.fit(X_train_tfidf, y_train_tfidf)
y_pred = model.predict(X_test_tfidf)
print('CountVectorizor: ')
evaluate(y_test_tfidf,predictions)

In [30]:
from xgboost import XGBRFClassifier
xgbrf = XGBRFClassifier(n_estimators=100, max_depth=5, random_state=42, verbosity=1)
model.fit(X_train_cv, y_train_cv)
y_pred = model.predict(X_test_cv)
print('CountVectorizor: ')
evaluate(y_test_cv,predictions)

CountVectorizor: 
Mean Squared Error:  0.46959646214296763
Precision:  0.7847146855543496
F1 Score:  0.7787638489106679
Accuracy:  0.7794791627428084
R2 Score:  0.11790014569512763


In [ ]:
model.fit(X_train_tfidf, y_train_tfidf)
y_pred = model.predict(X_test_tfidf)
print('CountVectorizor: ')
evaluate(y_test_tfidf,predictions)